<a href="https://colab.research.google.com/github/Michell203/mais-202/blob/main/MusicClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd